In [10]:
import pandas as pd
import numpy as np
import operator
import pickle
from collections import Counter,defaultdict
from sklearn.externals import joblib

In [76]:
test_df = pd.read_csv('test.csv')
pid = test_df['PassengerId']
test_df['Embarked'] = test_df['Embarked'].map({'S':1,'C':2,'Q':3})
test_df['Sex'] = test_df['Sex'].map({'male':1,'female':0})
test_df['Cabin'] = test_df['Cabin'].fillna(0)
test_df.loc[test_df['Cabin']!=0,'Cabin']=1
test_df = test_df.drop(['PassengerId','Age','SibSp','Ticket'],axis=1)
test_df.head(5)

,Pclass,Name,Sex,Parch,Fare,Cabin,Embarked
0,3,"Kelly, Mr. James",1,0,7.8292,0,3
1,3,"Wilkes, Mrs. James (Ellen Needs)",0,0,7.0000,0,1
2,2,"Myles, Mr. Thomas Francis",1,0,9.6875,0,3
3,3,"Wirz, Mr. Albert",1,0,8.6625,0,1
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,1,12.2875,0,1


In [77]:
split=0.0
for i in range(6):
    test_df['Fare'][operator.and_(test_df['Fare'] > split*100,test_df['Fare'] <= (split+1)*100)] = i
    split += 1
test_df.head(6)

C:\Users\HARIKRISHNAN\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Pclass,Name,Sex,Parch,Fare,Cabin,Embarked
0,3,"Kelly, Mr. James",1,0,0.0,0,3
1,3,"Wilkes, Mrs. James (Ellen Needs)",0,0,0.0,0,1
2,2,"Myles, Mr. Thomas Francis",1,0,0.0,0,3
3,3,"Wirz, Mr. Albert",1,0,0.0,0,1
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,1,0.0,0,1
5,3,"Svensson, Mr. Johan Cervin",1,0,0.0,0,1


In [78]:
with open('title_map.pkl','rb') as f:
    title_map = pickle.load(f)

In [79]:
test_df['Name'] = [title_map[i] if i in title_map.keys() else 0 for i in test_df.Name.str.split(' ').str[1]]
test_df.head(5)

,Pclass,Name,Sex,Parch,Fare,Cabin,Embarked
0,3,1,1,0,0.0,0,3
1,3,3,0,0,0.0,0,1
2,2,1,1,0,0.0,0,3
3,3,1,1,0,0.0,0,1
4,3,3,0,1,0.0,0,1


In [80]:
test_df[test_df['Fare'].isnull()]

,Pclass,Name,Sex,Parch,Fare,Cabin,Embarked
152,3,1,1,0,NaN,0,1


In [81]:
test_df['Fare'] = test_df['Fare'].fillna(0)

In [82]:
# Load models
svm = joblib.load('svm.pkl')
ada = joblib.load('ada.pkl')
mlp = joblib.load('mlp.pkl')

In [96]:
pred_values = test_df.values
votes = zip(mlp.predict(pred_values),ada.predict(pred_values),svm.predict(pred_values))
pred_labels = [Counter(vote).most_common(1)[0][0] for vote in votes]

In [109]:
pred_labels = mlp.predict(pred_values)

In [110]:
pred_df = pd.DataFrame(pid)
pred_df['Survived'] = pred_labels
pred_df.head(5)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [111]:
pred_df.to_csv('submission.csv',index=False)